In [ ]:
from banner import show_trivision_banner
print("🔁 Starting TriVision script...")  # Debug check
show_trivision_banner(animated=True, sound=True)

import cv2
import numpy as np
import os
from ultralytics import YOLO
from insightface.app import FaceAnalysis

# Load models
har_model = YOLO("C:/Users/rishi/OneDrive/Desktop/Trivision/models/HAR/best.pt")        # HAR detection
helmet_model = YOLO("C:/Users/rishi/OneDrive/Desktop/DeepLearning_Models/runs/detect/train19/weights/best.pt")    # Helmet detection
face_model = YOLO("yolov8n.pt")                                # Person detection for faces
app = FaceAnalysis(name="buffalo_l", providers=['CUDAExecutionProvider'])
app.prepare(ctx_id=0)

# Load known reference face embeddings
reference_dir = "C:/Users/rishi/OneDrive/Desktop/DeepLearning_Models/Face Detection/reference_images"
known_embeddings = []
known_names = []

for filename in os.listdir(reference_dir):
    if filename.lower().endswith(('.jpg', '.png')):
        img_path = os.path.join(reference_dir, filename)
        img = cv2.imread(img_path)
        faces = app.get(img)
        if faces:
            known_embeddings.append(np.array(faces[0].normed_embedding))
            known_names.append(os.path.splitext(filename)[0])
print("✅ Loaded reference faces:", known_names)

# Cosine similarity
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Start camera
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    annotated = frame.copy()

    # HAR Detection
    har_results = har_model(frame)
    annotated = har_results[0].plot(img=annotated)

    # Helmet Detection
    helmet_results = helmet_model(frame)
    annotated = helmet_results[0].plot(img=annotated)

    # Face Recognition
    face_results = face_model(frame, classes=[0])
    for det in face_results[0].boxes:
        x1, y1, x2, y2 = map(int, det.xyxy[0])
        person_crop = frame[y1:y2, x1:x2]

        faces = app.get(person_crop)
        for face in faces:
            emb = np.array(face.normed_embedding)
            name = "Unknown"
            max_sim = 0.0
            for i, known_emb in enumerate(known_embeddings):
                sim = cosine_similarity(emb, known_emb)
                if sim > 0.4 and sim > max_sim:
                    max_sim = sim
                    name = known_names[i]
            cv2.rectangle(annotated, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(annotated, f"{name} ({max_sim:.2f})", (x1, y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    # Show unified window
    cv2.imshow("TriVision - Jetson Inspection HUD (Press 'x' to Exit)", annotated)
    if cv2.waitKey(1) & 0xFF == ord("x"):
        break

cap.release()
cv2.destroyAllWindows()


pygame 2.6.1 (SDL 2.28.4, Python 3.10.18)
Hello from the pygame community. https://www.pygame.org/contribute.html
🔁 Starting TriVision script...


c:\Users\rishi\.conda\envs\Jetson\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for Jupyter
support
  warnings.warn('install "ipywidgets" for Jupyter support')

Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CUDAExecutionProvider': {'device_id': '0', 'has_user_compute_stream': '0', 'cudnn_conv1d_pad_to_nc1d': '0', 'user_compute_stream': '0', 'gpu_external_alloc': '0', 'gpu_mem_limit': '18446744073709551615', 'enable_cuda_graph': '0', 'gpu_external_free': '0', 'gpu_external_empty_cache': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'do_copy_in_default_stream': '1', 'cudnn_conv_use_max_workspace': '1', 'tunable_op_enable': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'prefer_nhwc': '0', 'use_ep_level_unified_stream': '0', 'use_tf32': '1', 'sdpa_kernel': '0', 'fuse_conv_bias': '0'}, 'CPUExecutionProvider': {}}
find model: C:\Users\rishi/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'

c:\Users\rishi\.conda\envs\Jetson\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


✅ Loaded reference faces: ['Ajitesh', 'Rahul']

0: 480x640 (no detections), 67.9ms
Speed: 3.5ms preprocess, 67.9ms inference, 23.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 NO-Hardhats, 41.0ms
Speed: 1.4ms preprocess, 41.0ms inference, 73.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 30.1ms
Speed: 1.6ms preprocess, 30.1ms inference, 27.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 27.5ms
Speed: 2.2ms preprocess, 27.5ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 NO-Hardhat, 17.9ms
Speed: 1.6ms preprocess, 17.9ms inference, 2.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 9.7ms
Speed: 1.2ms preprocess, 9.7ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)


c:\Users\rishi\.conda\envs\Jetson\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4



0: 480x640 (no detections), 20.2ms
Speed: 1.8ms preprocess, 20.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 NO-Hardhat, 16.8ms
Speed: 1.7ms preprocess, 16.8ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 21.6ms
Speed: 1.7ms preprocess, 21.6ms inference, 4.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 25.3ms
Speed: 1.3ms preprocess, 25.3ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 NO-Hardhat, 17.0ms
Speed: 1.5ms preprocess, 17.0ms inference, 2.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 11.5ms
Speed: 1.6ms preprocess, 11.5ms inference, 2.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 19.8ms
Speed: 1.2ms preprocess, 19.8ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 NO-Hardhat, 13.9ms
Speed: 2.5ms preprocess, 13.9ms inference, 2.9ms postpro